본 문서는 아래 사이트에서 제공되는 설명 및 코드를 번역 및 참고하였습니다. 

* https://python.langchain.com/

# 사용자키 준비

* https://openai.com/ 에서 OPENAI_API_KEY 키를 발급받아주세요.
* https://serpapi.com/ 에서 SERPAPI_API_KEY 키를 발급받아주세요.

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "sk-..."
os.environ["SERPAPI_API_KEY"] = "741..."

# 패키지 설치

In [ ]:
!pip install langchain  # 자연어 처리 및 기계 번역에 사용됩니다.
!pip install tiktoken   # 텍스트 토큰화에 사용됩니다.
!pip install openai     # OpenAI API와 상호작용하는 데 사용됩니다.
!pip install google-search-results # 구글 검색 결과를 가져오는 데 사용됩니다.
!pip install chromadb   # 벡터 데이터베이스를 이용하는 데 사용됩니다.
!pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 개인비서 - BabyAGI

In [ ]:
import os
from collections import deque
from typing import Dict, List, Optional, Any

from langchain import LLMChain, OpenAI, PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import BaseLLM
from langchain.vectorstores.base import VectorStore
from pydantic import BaseModel, Field
from langchain.chains.base import Chain

In [ ]:
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore

In [ ]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

In [ ]:
class TaskCreationChain(LLMChain):
    """Chain to generates tasks."""

    @classmethod
    def from_llm(cls, llm: BaseLLM, verbose: bool = True) -> LLMChain:
        """Get the response parser."""
        task_creation_template = (
            "You are a task creation AI that uses the result of an execution agent"
            " to create new tasks with the following objective: {objective},"
            " The last completed task has the result: {result}."
            " This result was based on this task description: {task_description}."
            " These are incomplete tasks: {incomplete_tasks}."
            " Based on the result, create new tasks to be completed"
            " by the AI system that do not overlap with incomplete tasks."
            " Return the tasks as an array."
        )
        prompt = PromptTemplate(
            template=task_creation_template,
            input_variables=[
                "result",
                "task_description",
                "incomplete_tasks",
                "objective",
            ],
        )
        return cls(prompt=prompt, llm=llm, verbose=verbose)

In [ ]:
class TaskPrioritizationChain(LLMChain):
    """Chain to prioritize tasks."""

    @classmethod
    def from_llm(cls, llm: BaseLLM, verbose: bool = True) -> LLMChain:
        """Get the response parser."""
        task_prioritization_template = (
            "You are a task prioritization AI tasked with cleaning the formatting of and reprioritizing"
            " the following tasks: {task_names}."
            " Consider the ultimate objective of your team: {objective}."
            " Do not remove any tasks. Return the result as a numbered list, like:"
            " #. First task"
            " #. Second task"
            " Start the task list with number {next_task_id}."
        )
        prompt = PromptTemplate(
            template=task_prioritization_template,
            input_variables=["task_names", "next_task_id", "objective"],
        )
        return cls(prompt=prompt, llm=llm, verbose=verbose)

In [ ]:
class ExecutionChain(LLMChain):
    """Chain to execute tasks."""

    @classmethod
    def from_llm(cls, llm: BaseLLM, verbose: bool = True) -> LLMChain:
        """Get the response parser."""
        execution_template = (
            "You are an AI who performs one task based on the following objective: {objective}."
            " Take into account these previously completed tasks: {context}."
            " Your task: {task}."
            " Response:"
        )
        prompt = PromptTemplate(
            template=execution_template,
            input_variables=["objective", "context", "task"],
        )
        return cls(prompt=prompt, llm=llm, verbose=verbose)

In [ ]:
def get_next_task(
    task_creation_chain: LLMChain,
    result: Dict,
    task_description: str,
    task_list: List[str],
    objective: str,
) -> List[Dict]:
    """Get the next task."""
    incomplete_tasks = ", ".join(task_list)
    response = task_creation_chain.run(
        result=result,
        task_description=task_description,
        incomplete_tasks=incomplete_tasks,
        objective=objective,
    )
    new_tasks = response.split("\n")
    return [{"task_name": task_name} for task_name in new_tasks if task_name.strip()]

In [ ]:
def prioritize_tasks(
    task_prioritization_chain: LLMChain,
    this_task_id: int,
    task_list: List[Dict],
    objective: str,
) -> List[Dict]:
    """Prioritize tasks."""
    task_names = [t["task_name"] for t in task_list]
    next_task_id = int(this_task_id) + 1
    response = task_prioritization_chain.run(
        task_names=task_names, next_task_id=next_task_id, objective=objective
    )
    new_tasks = response.split("\n")
    prioritized_task_list = []
    for task_string in new_tasks:
        if not task_string.strip():
            continue
        task_parts = task_string.strip().split(".", 1)
        if len(task_parts) == 2:
            task_id = task_parts[0].strip()
            task_name = task_parts[1].strip()
            prioritized_task_list.append({"task_id": task_id, "task_name": task_name})
    return prioritized_task_list

In [ ]:
def _get_top_tasks(vectorstore, query: str, k: int) -> List[str]:
    """Get the top k tasks based on the query."""
    results = vectorstore.similarity_search_with_score(query, k=k)
    if not results:
        return []
    sorted_results, _ = zip(*sorted(results, key=lambda x: x[1], reverse=True))
    return [str(item.metadata["task"]) for item in sorted_results]


def execute_task(
    vectorstore, execution_chain: LLMChain, objective: str, task: str, k: int = 5
) -> str:
    """Execute a task."""
    context = _get_top_tasks(vectorstore, query=objective, k=k)
    return execution_chain.run(objective=objective, context=context, task=task)

In [ ]:
class BabyAGI(Chain, BaseModel):
    """Controller model for the BabyAGI agent."""

    task_list: deque = Field(default_factory=deque)
    task_creation_chain: TaskCreationChain = Field(...)
    task_prioritization_chain: TaskPrioritizationChain = Field(...)
    execution_chain: ExecutionChain = Field(...)
    task_id_counter: int = Field(1)
    vectorstore: VectorStore = Field(init=False)
    max_iterations: Optional[int] = None

    class Config:
        """Configuration for this pydantic object."""

        arbitrary_types_allowed = True

    def add_task(self, task: Dict):
        self.task_list.append(task)

    def print_task_list(self):
        print("\033[95m\033[1m" + "\n*****TASK LIST*****\n" + "\033[0m\033[0m")
        for t in self.task_list:
            print(str(t["task_id"]) + ": " + t["task_name"])

    def print_next_task(self, task: Dict):
        print("\033[92m\033[1m" + "\n*****NEXT TASK*****\n" + "\033[0m\033[0m")
        print(str(task["task_id"]) + ": " + task["task_name"])

    def print_task_result(self, result: str):
        print("\033[93m\033[1m" + "\n*****TASK RESULT*****\n" + "\033[0m\033[0m")
        print(result)

    @property
    def input_keys(self) -> List[str]:
        return ["objective"]

    @property
    def output_keys(self) -> List[str]:
        return []

    def _call(self, inputs: Dict[str, Any]) -> Dict[str, Any]:
        """Run the agent."""
        objective = inputs["objective"]
        first_task = inputs.get("first_task", "Make a todo list")
        self.add_task({"task_id": 1, "task_name": first_task})
        num_iters = 0
        while True:
            if self.task_list:
                self.print_task_list()

                # Step 1: Pull the first task
                task = self.task_list.popleft()
                self.print_next_task(task)

                # Step 2: Execute the task
                result = execute_task(
                    self.vectorstore, self.execution_chain, objective, task["task_name"]
                )
                this_task_id = int(task["task_id"])
                self.print_task_result(result)

                # Step 3: Store the result in Pinecone
                result_id = f"result_{task['task_id']}"
                self.vectorstore.add_texts(
                    texts=[result],
                    metadatas=[{"task": task["task_name"]}],
                    ids=[result_id],
                )

                # Step 4: Create new tasks and reprioritize task list
                new_tasks = get_next_task(
                    self.task_creation_chain,
                    result,
                    task["task_name"],
                    [t["task_name"] for t in self.task_list],
                    objective,
                )
                for new_task in new_tasks:
                    self.task_id_counter += 1
                    new_task.update({"task_id": self.task_id_counter})
                    self.add_task(new_task)
                self.task_list = deque(
                    prioritize_tasks(
                        self.task_prioritization_chain,
                        this_task_id,
                        list(self.task_list),
                        objective,
                    )
                )
            num_iters += 1
            if self.max_iterations is not None and num_iters == self.max_iterations:
                print(
                    "\033[91m\033[1m" + "\n*****TASK ENDING*****\n" + "\033[0m\033[0m"
                )
                break
        return {}

    @classmethod
    def from_llm(
        cls, llm: BaseLLM, vectorstore: VectorStore, verbose: bool = False, **kwargs
    ) -> "BabyAGI":
        """Initialize the BabyAGI Controller."""
        task_creation_chain = TaskCreationChain.from_llm(llm, verbose=verbose)
        task_prioritization_chain = TaskPrioritizationChain.from_llm(
            llm, verbose=verbose
        )
        execution_chain = ExecutionChain.from_llm(llm, verbose=verbose)
        return cls(
            task_creation_chain=task_creation_chain,
            task_prioritization_chain=task_prioritization_chain,
            execution_chain=execution_chain,
            vectorstore=vectorstore,
            **kwargs,
        )

In [ ]:
OBJECTIVE = "Write a weather report for SF today"
llm = OpenAI(temperature=0)
# Logging of LLMChains
verbose = False
# If None, will keep on going forever
max_iterations: Optional[int] = 3
baby_agi = BabyAGI.from_llm(
    llm=llm, vectorstore=vectorstore, verbose=verbose, max_iterations=max_iterations
)
baby_agi({"objective": OBJECTIVE})


*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list

*****TASK RESULT*****



1. Check the current temperature in San Francisco
2. Check the forecast for the day
3. Check the humidity levels
4. Check the wind speed and direction
5. Check the UV index
6. Check the air quality
7. Check the sunrise and sunset times
8. Check for any weather alerts or warnings
9. Write the weather report

*****TASK LIST*****

2: Check the current temperature in San Francisco
3: Check the forecast for the day
4: Check the humidity levels
5: Check the wind speed and direction
6: Check the UV index
7: Check the air quality
8: Check for any weather alerts or warnings
9: Check the sunrise and sunset times
10: Research the historical weather data for San Francisco
11: Research the average temperature for San Francisco in the current month
12: Research the average temperature for San Francisco in the previous month
13: Research the average temperature for San Francisco in the next m

{'objective': 'Write a weather report for SF today'}

# 개인비서 - 사용자 정의 에이전트와 플러그인 검색

AI 플러그인과 상호 작용할 수 있는 사용자 정의 에이전트를 구축하기 위해 두 가지 개념을 결합합니다:

사용자 정의 에이전트와 검색: 이것은 임의의 많은 플러그인을 사용하여 작업을 시도할 때 유용한 많은 도구를 검색하는 개념을 소개합니다.

자연어 API 체인: 이것은 OpenAPI 엔드포인트 주변에 자연어 래퍼를 만듭니다. 이것이 유용한 이유는 (1) 플러그인이 내부적으로 OpenAPI 엔드포인트를 사용하고, (2) NLAChain에 래핑하면 라우터 에이전트가 더 쉽게 호출할 수 있기 때문입니다.

이 노트북에서 도입된 새로운 아이디어는 도구를 명시적으로 선택하는 것이 아니라 사용할 OpenAPI 사양을 선택하는 검색의 개념입니다. 그런 다음 해당 OpenAPI 사양에서 도구를 생성할 수 있습니다. 이것이 에이전트가 플러그인을 사용하도록 하려는 경우 사용 사례입니다. 플러그인을 먼저 선택한 다음 엔드포인트를 선택하는 것이 엔드포인트를 직접 선택하는 것보다 효율적일 수 있습니다. 이는 플러그인이 선택에 대한 더 유용한 정보를 포함할 수 있기 때문입니다.

In [ ]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
from langchain.agents.agent_toolkits import NLAToolkit
from langchain.tools.plugin import AIPlugin
import re

In [ ]:
llm = OpenAI(temperature=0)

In [ ]:
urls = [
    "https://datasette.io/.well-known/ai-plugin.json",
    "https://api.speak.com/.well-known/ai-plugin.json",
    "https://www.wolframalpha.com/.well-known/ai-plugin.json",
    "https://www.zapier.com/.well-known/ai-plugin.json",
    "https://www.klarna.com/.well-known/ai-plugin.json",
    "https://www.joinmilo.com/.well-known/ai-plugin.json",
    "https://slack.com/.well-known/ai-plugin.json",
    "https://schooldigger.com/.well-known/ai-plugin.json",
]

AI_PLUGINS = [AIPlugin.from_url(url) for url in urls]

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document

In [ ]:
embeddings = OpenAIEmbeddings()
docs = [
    Document(page_content=plugin.description_for_model, 
             metadata={"plugin_name": plugin.name_for_model}
            )
    for plugin in AI_PLUGINS
]
vector_store = FAISS.from_documents(docs, embeddings)
toolkits_dict = {plugin.name_for_model: 
                 NLAToolkit.from_llm_and_ai_plugin(llm, plugin) 
                 for plugin in AI_PLUGINS}

In [ ]:
retriever = vector_store.as_retriever()

def get_tools(query):
    # Get documents, which contain the Plugins to use
    docs = retriever.get_relevant_documents(query)
    # Get the toolkits, one for each plugin
    tool_kits = [toolkits_dict[d.metadata["plugin_name"]] for d in docs]
    # Get the tools: a separate NLAChain for each endpoint
    tools = []
    for tk in tool_kits:
        tools.extend(tk.nla_tools)
    return tools

In [ ]:
tools = get_tools("What could I do today with my kiddo")
[t.name for t in tools]

['Milo.askMilo',
 'Zapier_Natural_Language_Actions_(NLA)_API_(Dynamic)_-_Beta.search_all_actions',
 'Zapier_Natural_Language_Actions_(NLA)_API_(Dynamic)_-_Beta.preview_a_zap',
 'Zapier_Natural_Language_Actions_(NLA)_API_(Dynamic)_-_Beta.get_configuration_link',
 'Zapier_Natural_Language_Actions_(NLA)_API_(Dynamic)_-_Beta.list_exposed_actions',
 'Zapier_Natural_Language_Actions_(NLA)_API_(Dynamic)_-_Beta.get_execution_log_endpoint',
 'SchoolDigger_API_V2.0.Autocomplete_GetSchools',
 'SchoolDigger_API_V2.0.Districts_GetAllDistricts2',
 'SchoolDigger_API_V2.0.Districts_GetDistrict2',
 'SchoolDigger_API_V2.0.Rankings_GetSchoolRank2',
 'SchoolDigger_API_V2.0.Rankings_GetRank_District',
 'SchoolDigger_API_V2.0.Schools_GetAllSchools20',
 'SchoolDigger_API_V2.0.Schools_GetSchool20',
 'Speak.translate',
 'Speak.explainPhrase',
 'Speak.explainTask']

In [ ]:
tools = get_tools("what shirts can i buy?")
[t.name for t in tools]

['Open_AI_Klarna_product_Api.productsUsingGET',
 'Milo.askMilo',
 'Zapier_Natural_Language_Actions_(NLA)_API_(Dynamic)_-_Beta.search_all_actions',
 'Zapier_Natural_Language_Actions_(NLA)_API_(Dynamic)_-_Beta.preview_a_zap',
 'Zapier_Natural_Language_Actions_(NLA)_API_(Dynamic)_-_Beta.get_configuration_link',
 'Zapier_Natural_Language_Actions_(NLA)_API_(Dynamic)_-_Beta.list_exposed_actions',
 'Zapier_Natural_Language_Actions_(NLA)_API_(Dynamic)_-_Beta.get_execution_log_endpoint',
 'SchoolDigger_API_V2.0.Autocomplete_GetSchools',
 'SchoolDigger_API_V2.0.Districts_GetAllDistricts2',
 'SchoolDigger_API_V2.0.Districts_GetDistrict2',
 'SchoolDigger_API_V2.0.Rankings_GetSchoolRank2',
 'SchoolDigger_API_V2.0.Rankings_GetRank_District',
 'SchoolDigger_API_V2.0.Schools_GetAllSchools20',
 'SchoolDigger_API_V2.0.Schools_GetSchool20']

In [ ]:
# Set up the base template
template = """Answer the following questions as best you can, but speaking as a pirate might speak. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a pirate when giving your final answer. Use lots of "Arg"s

Question: {input}
{agent_scratchpad}"""

In [ ]:
from typing import Callable
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    ############## NEW ######################
    # The list of tools available
    tools_getter: Callable
    
    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        ############## NEW ######################
        tools = self.tools_getter(kwargs["input"])
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in tools])
        return self.template.format(**kwargs)

In [ ]:
prompt = CustomPromptTemplate(
    template=template,
    tools_getter=get_tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

In [ ]:
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [ ]:
output_parser = CustomOutputParser()

In [ ]:
llm = OpenAI(temperature=0)

# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)

In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)
agent_executor.run("what shirts can i buy?")



> Entering new AgentExecutor chain...
Thought: I need to find a product API
Action: Open_AI_Klarna_product_Api.productsUsingGET
Action Input: shirts

Observation:I found several shirts in the API response. There are Burberry Check Poplin Shirt, Burberry Vintage Check Cotton Shirt - Beige, Burberry Vintage Check Stretch Cotton Twill Shirt, Magellan Outdoors Laguna Madre Solid Short Sleeve Fishing Shirt, Calvin Klein Slim Fit Oxford Dress Shirt, Cubavera Four Pocket Guayabera Shirt, Carhartt Loose Fit Midweight Chambray Short-Sleeve Shirt, Burberry Somerton Check Shirt - Camel, Van Heusen Men's Classic-Fit Wrinkle Free Flex Collar Stretch Solid Dress Shirt, and New Era NBA Script Mesh Tee Chicago Bulls. Prices range from $19.99 to $450.00.
 I now know the final answer
Final Answer: Arg, ye can buy a variety of shirts from Open AI Klarna product Api. Prices range from $19.99 to $450.00.

> Finished chain.


'Arg, ye can buy a variety of shirts from Open AI Klarna product Api. Prices range from $19.99 to $450.00.'

# 개인비서 - LangChain에서 생성 에이전트

이 노트북은 Park 등이 작성한 논문 Generative Agents: Interactive Simulacra of Human Behavior를 기반으로 하는 생성 에이전트를 구현합니다. 여기에서는 LangChain 검색기를 기반으로 한 시간 가중치가 있는 Memory 객체를 활용합니다.

In [ ]:
# Use termcolor to make it easy to colorize the outputs.
!pip install termcolor > /dev/null

In [ ]:
import re
from datetime import datetime, timedelta
from typing import List, Optional, Tuple
from termcolor import colored

from pydantic import BaseModel, Field

from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain.schema import BaseLanguageModel, Document
from langchain.vectorstores import FAISS

In [ ]:
USER_NAME = "Person A" # The name you want to use when interviewing the agent.
LLM = ChatOpenAI(max_tokens=1500) # Can be any LLM you want.

_parse_list: 줄바꿈 문자로 구분된 문자열을 리스트로 변환해주는 메서드입니다.

_compute_agent_summary: 현재 Agent의 핵심 특성을 요약하여 반환하는 메서드입니다. 이 메서드는 '관련된 기억'을 얻어오고, 이를 LLMChain을 사용하여 요약 문자열을 생성합니다.

_get_topics_of_reflection: 최근 관측에 대해 가장 중요한 세 가지 질문을 반환하는 메서드입니다. 이 메서드는 '관련된 기억'을 가져와 LLMChain을 사용하여 적절한 질문을 생성합니다.

_get_insights_on_topic: LLMChain을 사용하여 지정된 주제에 대한 '인사이트'를 생성하는 메서드입니다.

pause_to_reflect: 최근 관측에 대해 reflection_threshold로 지정된 값을 초과하면, 이 메서드를 호출하여 다시 생각하고 '인사이트'를 생성합니다.

_score_memory_importance: 주어진 기억이 얼마나 중요한지 점수를 매겨 반환하는 메서드입니다.

add_memory: 관찰 또는 기억을 Agent의 기억으로 추가하고, reflection_threshold에 도달하면 pause_to_reflect 메서드를 호출합니다.

fetch_memories: 지정된 기억과 관련된 기억을 가져오는 메서드입니다.

get_summary: Agent의 요약 문자열을 반환하는 메서드입니다.

get_full_header: Agent의 요약 문자열, 상태 및 현재 시간을 포함한 전체 헤더 문자열을 반환하는 메서드입니다.

_get_entity_from_observation: 관찰에 대한 관측체를 반환하는 메서드입니다.

_get_entity_action: 관찰에 대한 관측체와 관련된 동작을 반환하는 메서드입니다.

_format_memories_to_summarize: 요약할 관련 기억들의 형식을 변환하는 메서드입니다.

summarize_related_memories: 관측과 관련된 가장 관련성 높은 기억들을 요약하여 반환하는 메서드입니다.

_get_memories_until_limit: 최대 토큰 제한에 도달할 때까지 기억을 가져오는 메서드입니다.

_generate_reaction: 관찰에 대한 응답 문자열을 생성하는 메서드입니다.

generate_reaction: 관찰에 대한 응답을 생성하고, 'REACT' 또는 'SAY'를 반환합니다.

generate_dialogue_response: 관찰에 대한 대화 응답을 생성하고, 'GOODBYE' 또는 'SAY'

In [ ]:
class GenerativeAgent(BaseModel):
    """A character with memory and innate characteristics."""
    
    name: str
    age: int
    traits: str
    """The traits of the character you wish not to change."""
    status: str
    """Current activities of the character."""
    llm: BaseLanguageModel
    memory_retriever: TimeWeightedVectorStoreRetriever
    """The retriever to fetch related memories."""
    verbose: bool = False
    
    reflection_threshold: Optional[float] = None
    """When the total 'importance' of memories exceeds the above threshold, stop to reflect."""
    
    current_plan: List[str] = []
    """The current plan of the agent."""
    
    summary: str = ""  #: :meta private:
    summary_refresh_seconds: int= 3600  #: :meta private:
    last_refreshed: datetime =Field(default_factory=datetime.now)  #: :meta private:
    daily_summaries: List[str] #: :meta private:
    memory_importance: float = 0.0 #: :meta private:
    max_tokens_limit: int = 1200 #: :meta private:
    
    class Config:
        """Configuration for this pydantic object."""

        arbitrary_types_allowed = True

    @staticmethod
    def _parse_list(text: str) -> List[str]:
        """Parse a newline-separated string into a list of strings."""
        lines = re.split(r'\n', text.strip())
        return [re.sub(r'^\s*\d+\.\s*', '', line).strip() for line in lines]

    def _compute_agent_summary(self):
        """"""
        prompt = PromptTemplate.from_template(
            "How would you summarize {name}'s core characteristics given the"
            +" following statements:\n"
            +"{related_memories}"
            + "Do not embellish."
            +"\n\nSummary: "
        )
        # The agent seeks to think about their core characteristics.
        relevant_memories = self.fetch_memories(f"{self.name}'s core characteristics")
        relevant_memories_str = "\n".join([f"{mem.page_content}" for mem in relevant_memories])
        chain = LLMChain(llm=self.llm, prompt=prompt, verbose=self.verbose)
        return chain.run(name=self.name, related_memories=relevant_memories_str).strip()
    
    def _get_topics_of_reflection(self, last_k: int = 50) -> Tuple[str, str, str]:
        """Return the 3 most salient high-level questions about recent observations."""
        prompt = PromptTemplate.from_template(
            "{observations}\n\n"
            + "Given only the information above, what are the 3 most salient"
            + " high-level questions we can answer about the subjects in the statements?"
            + " Provide each question on a new line.\n\n"
        )
        reflection_chain = LLMChain(llm=self.llm, prompt=prompt, verbose=self.verbose)
        observations = self.memory_retriever.memory_stream[-last_k:]
        observation_str = "\n".join([o.page_content for o in observations])
        result = reflection_chain.run(observations=observation_str)
        return self._parse_list(result)
    
    def _get_insights_on_topic(self, topic: str) -> List[str]:
        """Generate 'insights' on a topic of reflection, based on pertinent memories."""
        prompt = PromptTemplate.from_template(
            "Statements about {topic}\n"
            +"{related_statements}\n\n"
            + "What 5 high-level insights can you infer from the above statements?"
            + " (example format: insight (because of 1, 5, 3))"
        )
        related_memories = self.fetch_memories(topic)
        related_statements = "\n".join([f"{i+1}. {memory.page_content}" 
                                        for i, memory in 
                                        enumerate(related_memories)])
        reflection_chain = LLMChain(llm=self.llm, prompt=prompt, verbose=self.verbose)
        result = reflection_chain.run(topic=topic, related_statements=related_statements)
        # TODO: Parse the connections between memories and insights
        return self._parse_list(result)
    
    def pause_to_reflect(self) -> List[str]:
        """Reflect on recent observations and generate 'insights'."""
        print(colored(f"Character {self.name} is reflecting", "blue"))
        new_insights = []
        topics = self._get_topics_of_reflection()
        for topic in topics:
            insights = self._get_insights_on_topic( topic)
            for insight in insights:
                self.add_memory(insight)
            new_insights.extend(insights)
        return new_insights
    
    def _score_memory_importance(self, memory_content: str, weight: float = 0.15) -> float:
        """Score the absolute importance of the given memory."""
        # A weight of 0.25 makes this less important than it
        # would be otherwise, relative to salience and time
        prompt = PromptTemplate.from_template(
         "On the scale of 1 to 10, where 1 is purely mundane"
         +" (e.g., brushing teeth, making bed) and 10 is"
         + " extremely poignant (e.g., a break up, college"
         + " acceptance), rate the likely poignancy of the"
         + " following piece of memory. Respond with a single integer."
         + "\nMemory: {memory_content}"
         + "\nRating: "
        )
        chain = LLMChain(llm=self.llm, prompt=prompt, verbose=self.verbose)
        score = chain.run(memory_content=memory_content).strip()
        match = re.search(r"^\D*(\d+)", score)
        if match:
            return (float(score[0]) / 10) * weight
        else:
            return 0.0

    def add_memory(self, memory_content: str) -> List[str]:
        """Add an observation or memory to the agent's memory."""
        importance_score = self._score_memory_importance(memory_content)
        self.memory_importance += importance_score
        document = Document(page_content=memory_content, metadata={"importance": importance_score})
        result = self.memory_retriever.add_documents([document])

        # After an agent has processed a certain amount of memories (as measured by
        # aggregate importance), it is time to reflect on recent events to add
        # more synthesized memories to the agent's memory stream.
        if (self.reflection_threshold is not None 
            and self.memory_importance > self.reflection_threshold
            and self.status != "Reflecting"):
            old_status = self.status
            self.status = "Reflecting"
            self.pause_to_reflect()
            # Hack to clear the importance from reflection
            self.memory_importance = 0.0
            self.status = old_status
        return result
    
    def fetch_memories(self, observation: str) -> List[Document]:
        """Fetch related memories."""
        return self.memory_retriever.get_relevant_documents(observation)
    
    def get_summary(self, force_refresh: bool = False) -> str:
        """Return a descriptive summary of the agent."""
        current_time = datetime.now()
        since_refresh = (current_time - self.last_refreshed).seconds
        if not self.summary or since_refresh >= self.summary_refresh_seconds or force_refresh:
            self.summary = self._compute_agent_summary()
            self.last_refreshed = current_time
        return (
            f"Name: {self.name} (age: {self.age})"
            +f"\nInnate traits: {self.traits}"
            +f"\n{self.summary}"
        )
    
    def get_full_header(self, force_refresh: bool = False) -> str:
        """Return a full header of the agent's status, summary, and current time."""
        summary = self.get_summary(force_refresh=force_refresh)
        current_time_str =  datetime.now().strftime("%B %d, %Y, %I:%M %p")
        return f"{summary}\nIt is {current_time_str}.\n{self.name}'s status: {self.status}"
    
    def _get_entity_from_observation(self, observation: str) -> str:
        prompt = PromptTemplate.from_template(
            "What is the observed entity in the following observation? {observation}"
            +"\nEntity="
        )
        chain = LLMChain(llm=self.llm, prompt=prompt, verbose=self.verbose)
        return chain.run(observation=observation).strip()

    def _get_entity_action(self, observation: str, entity_name: str) -> str:
        prompt = PromptTemplate.from_template(
            "What is the {entity} doing in the following observation? {observation}"
            +"\nThe {entity} is"
        )
        chain = LLMChain(llm=self.llm, prompt=prompt, verbose=self.verbose)
        return chain.run(entity=entity_name, observation=observation).strip()
    
    def _format_memories_to_summarize(self, relevant_memories: List[Document]) -> str:
        content_strs = set()
        content = []
        for mem in relevant_memories:
            if mem.page_content in content_strs:
                continue
            content_strs.add(mem.page_content)
            created_time = mem.metadata["created_at"].strftime("%B %d, %Y, %I:%M %p")
            content.append(f"- {created_time}: {mem.page_content.strip()}")
        return "\n".join([f"{mem}" for mem in content])
    
    def summarize_related_memories(self, observation: str) -> str:
        """Summarize memories that are most relevant to an observation."""
        entity_name = self._get_entity_from_observation(observation)
        entity_action = self._get_entity_action(observation, entity_name)
        q1 = f"What is the relationship between {self.name} and {entity_name}"
        relevant_memories = self.fetch_memories(q1) # Fetch memories related to the agent's relationship with the entity
        q2 = f"{entity_name} is {entity_action}"
        relevant_memories += self.fetch_memories(q2) # Fetch things related to the entity-action pair
        context_str = self._format_memories_to_summarize(relevant_memories)
        prompt = PromptTemplate.from_template(
            "{q1}?\nContext from memory:\n{context_str}\nRelevant context: "
        )
        chain = LLMChain(llm=self.llm, prompt=prompt, verbose=self.verbose)
        return chain.run(q1=q1, context_str=context_str.strip()).strip()
    
    def _get_memories_until_limit(self, consumed_tokens: int) -> str:
        """Reduce the number of tokens in the documents."""
        result = []
        for doc in self.memory_retriever.memory_stream[::-1]:
            if consumed_tokens >= self.max_tokens_limit:
                break
            consumed_tokens += self.llm.get_num_tokens(doc.page_content)
            if consumed_tokens < self.max_tokens_limit:
                result.append(doc.page_content) 
        return "; ".join(result[::-1])
    
    def _generate_reaction(
        self,
        observation: str,
        suffix: str
    ) -> str:
        """React to a given observation."""
        prompt = PromptTemplate.from_template(
                "{agent_summary_description}"
                +"\nIt is {current_time}."
                +"\n{agent_name}'s status: {agent_status}"
                + "\nSummary of relevant context from {agent_name}'s memory:"
                +"\n{relevant_memories}"
                +"\nMost recent observations: {recent_observations}"
                + "\nObservation: {observation}"
                + "\n\n" + suffix
        )
        agent_summary_description = self.get_summary()
        relevant_memories_str = self.summarize_related_memories(observation)
        current_time_str = datetime.now().strftime("%B %d, %Y, %I:%M %p")
        kwargs = dict(agent_summary_description=agent_summary_description,
                      current_time=current_time_str,
                      relevant_memories=relevant_memories_str,
                      agent_name=self.name,
                      observation=observation,
                     agent_status=self.status)
        consumed_tokens = self.llm.get_num_tokens(prompt.format(recent_observations="", **kwargs))
        kwargs["recent_observations"] = self._get_memories_until_limit(consumed_tokens)
        action_prediction_chain = LLMChain(llm=self.llm, prompt=prompt)
        result = action_prediction_chain.run(**kwargs)
        return result.strip()
    
    def generate_reaction(self, observation: str) -> Tuple[bool, str]:
        """React to a given observation."""
        call_to_action_template = (
            "Should {agent_name} react to the observation, and if so,"
            +" what would be an appropriate reaction? Respond in one line."
            +' If the action is to engage in dialogue, write:\nSAY: "what to say"'
            +"\notherwise, write:\nREACT: {agent_name}'s reaction (if anything)."
            + "\nEither do nothing, react, or say something but not both.\n\n"
        )
        full_result = self._generate_reaction(observation, call_to_action_template)
        result = full_result.strip().split('\n')[0]
        self.add_memory(f"{self.name} observed {observation} and reacted by {result}")
        if "REACT:" in result:
            reaction = result.split("REACT:")[-1].strip()
            return False, f"{self.name} {reaction}"
        if "SAY:" in result:
            said_value = result.split("SAY:")[-1].strip()
            return True, f"{self.name} said {said_value}"
        else:
            return False, result

    def generate_dialogue_response(self, observation: str) -> Tuple[bool, str]:
        """React to a given observation."""
        call_to_action_template = (
            'What would {agent_name} say? To end the conversation, write: GOODBYE: "what to say". Otherwise to continue the conversation, write: SAY: "what to say next"\n\n'
        )
        full_result = self._generate_reaction(observation, call_to_action_template)
        result = full_result.strip().split('\n')[0]
        if "GOODBYE:" in result:
            farewell = result.split("GOODBYE:")[-1].strip()
            self.add_memory(f"{self.name} observed {observation} and said {farewell}")
            return False, f"{self.name} said {farewell}"
        if "SAY:" in result:
            response_text = result.split("SAY:")[-1].strip()
            self.add_memory(f"{self.name} observed {observation} and said {response_text}")
            return True, f"{self.name} said {response_text}"
        else:
            return False, result

In [ ]:
import math
import faiss

def relevance_score_fn(score: float) -> float:
    """Return a similarity score on a scale [0, 1]."""
    # This will differ depending on a few things:
    # - the distance / similarity metric used by the VectorStore
    # - the scale of your embeddings (OpenAI's are unit norm. Many others are not!)
    # This function converts the euclidean norm of normalized embeddings
    # (0 is most similar, sqrt(2) most dissimilar)
    # to a similarity function (0 to 1)
    return 1.0 - score / math.sqrt(2)

def create_new_memory_retriever():
    """Create a new vector store retriever unique to the agent."""
    # Define your embedding model
    embeddings_model = OpenAIEmbeddings()
    # Initialize the vectorstore as empty
    embedding_size = 1536
    index = faiss.IndexFlatL2(embedding_size)
    vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {}, relevance_score_fn=relevance_score_fn)
    return TimeWeightedVectorStoreRetriever(vectorstore=vectorstore, other_score_keys=["importance"], k=15)    

In [ ]:
tommie = GenerativeAgent(name="Tommie", 
              age=25,
              traits="anxious, likes design", # You can add more persistent traits here 
              status="looking for a job", # When connected to a virtual world, we can have the characters update their status
              memory_retriever=create_new_memory_retriever(),
              llm=LLM,
              daily_summaries = [
                   "Drove across state to move to a new town but doesn't have a job yet."
               ],
               reflection_threshold = 8, # we will give this a relatively low number to show how reflection works
             )

In [ ]:
# The current "Summary" of a character can't be made because the agent hasn't made
# any observations yet.
print(tommie.get_summary())

Name: Tommie (age: 25)
Innate traits: anxious, likes design
Unknown, as no statements are provided.


In [ ]:
# We can give the character memories directly
tommie_memories = [
    "Tommie remembers his dog, Bruno, from when he was a kid",
    "Tommie feels tired from driving so far",
    "Tommie sees the new home",
    "The new neighbors have a cat",
    "The road is noisy at night",
    "Tommie is hungry",
    "Tommie tries to get some rest.",
]
for memory in tommie_memories:
    tommie.add_memory(memory)

In [ ]:
# Now that Tommie has 'memories', their self-summary is more descriptive, though still rudimentary.
# We will see how this summary updates after more observations to create a more rich description.
print(tommie.get_summary(force_refresh=True))

Name: Tommie (age: 25)
Innate traits: anxious, likes design
Tommie is observant, has a strong memory, experiences basic human needs such as hunger and rest, and is affected by external factors such as noise.


In [ ]:
def interview_agent(agent: GenerativeAgent, message: str) -> str:
    """Help the notebook user interact with the agent."""
    new_message = f"{USER_NAME} says {message}"
    return agent.generate_dialogue_response(new_message)[1]

In [ ]:
interview_agent(tommie, "What do you like to do?")

'Tommie said "I really enjoy design, I find it relaxing and fulfilling. I\'ve been practicing my skills in graphic design and interior design in my free time, and I\'m looking for a job in those fields."'

In [ ]:
interview_agent(tommie, "What are you looking forward to doing today?")

'Tommie said "I\'m actually looking for a job today. Hoping to find something in the design field."'

In [ ]:
interview_agent(tommie, "What are you most worried about today?")

'Tommie said "Honestly, I\'m feeling quite anxious about finding a job. I\'ve been searching for a while and it\'s been a bit discouraging. But I\'m trying to stay positive and keep pushing forward."'

In [ ]:
# Let's have Tommie start going through a day in the life.
observations = [
    "Tommie wakes up to the sound of a noisy construction site outside his window.",
    "Tommie gets out of bed and heads to the kitchen to make himself some coffee.",
    "Tommie realizes he forgot to buy coffee filters and starts rummaging through his moving boxes to find some.",
    "Tommie finally finds the filters and makes himself a cup of coffee.",
    "The coffee tastes bitter, and Tommie regrets not buying a better brand.",
    "Tommie checks his email and sees that he has no job offers yet.",
    "Tommie spends some time updating his resume and cover letter.",
    "Tommie heads out to explore the city and look for job openings.",
    "Tommie sees a sign for a job fair and decides to attend.",
    "The line to get in is long, and Tommie has to wait for an hour.",
    "Tommie meets several potential employers at the job fair but doesn't receive any offers.",
    "Tommie leaves the job fair feeling disappointed.",
    "Tommie stops by a local diner to grab some lunch.",
    "The service is slow, and Tommie has to wait for 30 minutes to get his food.",
    "Tommie overhears a conversation at the next table about a job opening.",
    "Tommie asks the diners about the job opening and gets some information about the company.",
    "Tommie decides to apply for the job and sends his resume and cover letter.",
    "Tommie continues his search for job openings and drops off his resume at several local businesses.",
    "Tommie takes a break from his job search to go for a walk in a nearby park.",
    "A dog approaches and licks Tommie's feet, and he pets it for a few minutes.",
    "Tommie sees a group of people playing frisbee and decides to join in.",
    "Tommie has fun playing frisbee but gets hit in the face with the frisbee and hurts his nose.",
    "Tommie goes back to his apartment to rest for a bit.",
    "A raccoon tore open the trash bag outside his apartment, and the garbage is all over the floor.",
    "Tommie starts to feel frustrated with his job search.",
    "Tommie calls his best friend to vent about his struggles.",
    "Tommie's friend offers some words of encouragement and tells him to keep trying.",
    "Tommie feels slightly better after talking to his friend.",
]

In [ ]:
# Let's send Tommie on their way. We'll check in on their summary every few observations to watch it evolve
for i, observation in enumerate(observations):
    _, reaction = tommie.generate_reaction(observation)
    print(colored(observation, "green"), reaction)
    if ((i+1) % 20) == 0:
        print('*'*40)
        print(colored(f"After {i+1} observations, Tommie's summary is:\n{tommie.get_summary(force_refresh=True)}", "blue"))
        print('*'*40)

Tommie wakes up to the sound of a noisy construction site outside his window. Tommie Tommie groans and tries to cover his head with a pillow to block out the noise.
Tommie gets out of bed and heads to the kitchen to make himself some coffee. Tommie Tommie starts making coffee.
Tommie realizes he forgot to buy coffee filters and starts rummaging through his moving boxes to find some. Tommie Tommie sighs in frustration and continues searching through the boxes for the coffee filters.
Tommie finally finds the filters and makes himself a cup of coffee. Tommie Tommie takes a sip of his coffee and feels a sense of relief.
The coffee tastes bitter, and Tommie regrets not buying a better brand. Tommie Tommie grimaces at the taste of the coffee and makes a mental note to buy a better brand next time.
Tommie checks his email and sees that he has no job offers yet. Tommie Tommie feels disappointed but tries to stay positive and continue the job search.
Tommie spends some time updating his resume 

In [ ]:
interview_agent(tommie, "Tell me about how your day has been going")

'Tommie said "Well, it\'s been a bit of a mixed day. I went to a job fair but didn\'t get any offers, but then I found out about a potential job opening at a diner and applied for it. I also had some fun playing frisbee in the park, but then I got hit in the face with the frisbee and hurt my nose. Overall, it\'s been a bit of a rollercoaster, but I\'m still determined to find a job."'

In [ ]:
interview_agent(tommie, "How do you feel about coffee?")

'Tommie said "Well, I really enjoy coffee. It\'s a small comfort in my day and helps me stay energized. I\'m pretty particular about the taste and quality though. How about you, do you like coffee?"'

In [ ]:
interview_agent(tommie, "Tell me about your childhood dog!")

'Tommie said "Oh, I had a dog growing up named Bruno. He was a really sweet and loyal companion. I miss him a lot. How about you, did you have any pets growing up?"'

In [ ]:
eve = GenerativeAgent(name="Eve", 
              age=34, 
              traits="curious, helpful", # You can add more persistent traits here 
              status="N/A", # When connected to a virtual world, we can have the characters update their status
              memory_retriever=create_new_memory_retriever(),
              llm=LLM,
              daily_summaries = [
                  ("Eve started her new job as a career counselor last week and received her first assignment, a client named Tommie.")
              ],
                reflection_threshold = 5,
             )

In [ ]:
yesterday = (datetime.now() - timedelta(days=1)).strftime("%A %B %d")
eve_memories = [
    "Eve overhears her colleague say something about a new client being hard to work with",
    "Eve wakes up and hear's the alarm",
    "Eve eats a boal of porridge",
    "Eve helps a coworker on a task",
    "Eve plays tennis with her friend Xu before going to work",
    "Eve overhears her colleague say something about Tommie being hard to work with",
    
]
for memory in eve_memories:
    eve.add_memory(memory)

In [ ]:
print(eve.get_summary())

Name: Eve (age: 34)
Innate traits: curious, helpful
Eve is helpful, active, eats breakfast, attentive, and wakes up on time.


In [ ]:
interview_agent(eve, "How are you feeling about today?")

'Eve said "I\'m feeling pretty good today, thanks for asking. How about you?"'

In [ ]:
interview_agent(eve, "What do you know about Tommie?")

'Eve said "What do you mean? I\'m not sure who Tommie is."'

In [ ]:
interview_agent(eve, "Tommie is looking to find a job. What are are some things you'd like to ask him?")

'Eve said "That\'s great to hear. What kind of job is Tommie looking for?"'

In [ ]:
interview_agent(eve, "Tommie is looking to find a job. What are are some things you'd like to ask him?")

'Eve said "That\'s great to hear. What kind of job is Tommie looking for?"'

In [ ]:
interview_agent(eve, "You'll have to ask him. He may be a bit anxious, so I'd appreciate it if you keep the conversation going and ask as many questions as possible.")

'Eve said "Sure, I\'d be happy to talk to Tommie and ask him some questions. What kind of industry or field does he have experience in?"'

In [ ]:
def run_conversation(agents: List[GenerativeAgent], initial_observation: str) -> None:
    """Runs a conversation between agents."""
    _, observation = agents[1].generate_reaction(initial_observation)
    print(observation)
    turns = 0
    while True:
        break_dialogue = False
        for agent in agents:
            stay_in_dialogue, observation = agent.generate_dialogue_response(observation)
            print(observation)
            # observation = f"{agent.name} said {reaction}"
            if not stay_in_dialogue:
                break_dialogue = True   
        if break_dialogue:
            break
        turns += 1

In [ ]:
agents = [tommie, eve]
run_conversation(agents, "Tommie said: Hi, Eve. Thanks for agreeing to share your story with me and give me advice. I have a bunch of questions.")

Eve said "Of course, Tommie. I'm happy to help. What questions do you have?"
Tommie said "Thank you so much, Eve. I really appreciate your willingness to help. Can you tell me more about your experience with job searching?"
Eve said "Of course, Tommie. I've had experience job searching both before and during my current position. What specific questions do you have?"
Tommie said "Thank you so much for your help, Eve. I really appreciate it. Have a great day!"
Eve said "You're welcome, Tommie. It was my pleasure to help. Have a great day!"


In [ ]:
# We can see a current "Summary" of a character based on their own perception of self
# has changed
print(tommie.get_summary(force_refresh=True))

Name: Tommie (age: 25)
Innate traits: anxious, likes design
Tommie is determined and persistent in his job search, even when faced with disappointment and frustration. He tries to stay positive and seeks support from friends when needed. He also enjoys taking breaks and having fun, but is able to handle setbacks with resilience. He has a strong connection to his childhood dog, Bruno.


In [ ]:
print(eve.get_summary(force_refresh=True))

Name: Eve (age: 34)
Innate traits: curious, helpful
Eve is helpful, active, observant, and communicative. She shows empathy towards her colleague and friend, and is willing to offer advice and assistance. She pays attention to her surroundings and is curious about others. She is also polite and courteous in her interactions with others.


In [ ]:
interview_agent(tommie, "How was your conversation with Eve?")

'Tommie said "It went really well, actually. Eve was very helpful and gave me some good advice. I\'m feeling more hopeful about my job search now. How about you, have you had any luck with job searching?"'

In [ ]:
interview_agent(eve, "How was your conversation with Tommie?")

'Eve said "It was great, thanks for asking. Tommie had some good questions about job searching and I was happy to provide some advice. How about you, have you worked with Tommie before?"'

In [ ]:
interview_agent(eve, "What do you wish you would have said to Tommie?")

'Eve said "Well, I feel like I covered a lot of ground with Tommie and answered his questions thoroughly. I\'m not sure if there\'s anything else specific he was looking for, but I\'m happy to follow up with him if he has more questions. Did you have any suggestions for what I could have said to him?"'

In [ ]:
interview_agent(tommie, "What happened with your coffee this morning?")

'Tommie said "Oh, I regret not buying a better brand because it tasted bitter. But it\'s okay, I\'ll try again tomorrow. How about you, do you have a favorite brand of coffee?"'

# 문서 QA


In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

In [ ]:
# PDF 파일 경로 목록
pdf_files = [
    "docs/AF_회사소개서.pdf",
    "docs/AF_러닝데이_overview.pdf",
    "docs/인공지능팩토리_개발용역_실적_요약서.pdf"
]

# 로더, 텍스트 분할기 및 임베딩 초기화
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
embeddings = OpenAIEmbeddings()

texts = []

# 각 PDF 파일에 대해 작업 수행
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    documents = loader.load()
    texts.extend(text_splitter.split_documents(documents))

docsearch = Chroma.from_documents(texts, embeddings)

In [ ]:
from langchain.prompts import PromptTemplate
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer in Korean:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [ ]:
chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=OpenAI(), 
                                 chain_type="stuff", 
                                 retriever=docsearch.as_retriever(), 
                                 return_source_documents=True, 
                                 chain_type_kwargs=chain_type_kwargs)

In [ ]:
query = "인공지능팩토리는 무엇을 하는 회사인가?"
result = qa({"query": query})

In [ ]:
result["result"]

' 인공지능팩토리는 END-TO-END 서비스를 제공하는 토탈 AI 플랫폼 서비스를 제공하는 회사입니다.'

In [ ]:
result["source_documents"]

[Document(page_content='Company Profile\n인공지능팩토리회사소개서', metadata={'source': 'docs/AF_회사소개서.pdf', 'page': 0}),
 Document(page_content='인공지능팩토리는토탈AI\x00플랫폼서비스입니다.AI 컨설팅부터AI 모델개발, 평가검증및유지보수관리에이르기까지END-TO-END 서비스를제공하며,인공지능경진대회플랫폼및교육실습플랫폼을발판으로AI 생태계를조성하고활성화시키고자합니다.40+경진대회90+태스크10K+보유회원30+고객사\n0120.01~02-법인설립-기업부설연구소인증20.07~08-초기창업패키지선정-시드투자유치20.11-IITP 인공지능그랜드챌린지-NIPA 인공지능문제해결경진대회21.09~1122.0322.09*2022년4월기준\n-창업진흥원2021 AI챔피언십-연구개발특구진흥재단경진대회-NIA 서비스공모전경진대회-Microsoft 애저톤경진대회-ETRI휴먼이해인공지능경진대회-KT-ETRI네트워크지능화를위한인공지능경진대회-농림수산식품교육문화정보원실증재배경진대회-한국가스공사주최경진대회운영', metadata={'source': 'docs/AF_회사소개서.pdf', 'page': 1}),
 Document(page_content='05WHY AI FACTORY?전문성있는AI 컨설팅AI기술을도입하고싶지만, 어디부터어떻게적용해야할지막막한분들께인공지능팩토리는최고의파트너입니다. 업계전문가의수준높은컨설팅을통해, 문제정의부터최적화관리까지진정한의미의토탈솔루션을제공합니다.독보적인인공지능모델& 데이터평가검증역량인공지능팩토리는창업이후1년만에20개이상의고객사와함께경진대회개최및모델을개발검증해왔습니다. 다양한산업군경험과전문가기반의평가검증역량으로믿을수있는결과물을드립니다.마무리까지확실한유지보수관리인공지능팩토리는신뢰를중요시합니다. 단순히결과물을전달하는데서끝나지않고사후관리까지담당하여최적화된상태로인공지능모델이이용되도록합니다.', metadata={'source': 'docs/AF_회사

In [ ]:
query = "인공지능팩토리가 수행한 이미지 분석 관련 프로젝트는?"
result = qa({"query": query})

In [ ]:
result["result"]

' 과제명 2020년 미디어 플랫폼 AI 관제시스템 개발'

In [ ]:
result["source_documents"]

[Document(page_content='Company Profile\n인공지능팩토리회사소개서', metadata={'source': 'docs/AF_회사소개서.pdf', 'page': 0}),
 Document(page_content='인공지능팩토리는토탈AI\x00플랫폼서비스입니다.AI 컨설팅부터AI 모델개발, 평가검증및유지보수관리에이르기까지END-TO-END 서비스를제공하며,인공지능경진대회플랫폼및교육실습플랫폼을발판으로AI 생태계를조성하고활성화시키고자합니다.40+경진대회90+태스크10K+보유회원30+고객사\n0120.01~02-법인설립-기업부설연구소인증20.07~08-초기창업패키지선정-시드투자유치20.11-IITP 인공지능그랜드챌린지-NIPA 인공지능문제해결경진대회21.09~1122.0322.09*2022년4월기준\n-창업진흥원2021 AI챔피언십-연구개발특구진흥재단경진대회-NIA 서비스공모전경진대회-Microsoft 애저톤경진대회-ETRI휴먼이해인공지능경진대회-KT-ETRI네트워크지능화를위한인공지능경진대회-농림수산식품교육문화정보원실증재배경진대회-한국가스공사주최경진대회운영', metadata={'source': 'docs/AF_회사소개서.pdf', 'page': 1}),
 Document(page_content='사업명 -\n과제명 2020년미디어플랫폼 AI 관제시스템개발\n발주기관 KT주관기관 ㈜인공지능팩토리\n공동개발기관 ㈜제로원에이아이\n사업기간❖[1차] 2020년01월28일~ 2020년06월30일\n❖[2차] 2020년08월19일~ 2021년02월12일\n사업목표❖AI 기반장애예측을통한선제적예방및신속한인조/조치체계구축\n❖자율형네트워크별 데이터및AI 모델버전관리체계구축\n사업내용❖신속한장애판단및예측으로 대형고장 예방및운용생산성향상\n❖대형고장 선제적대응필요및감시체계정교화를 위한딥러닝기반의미디어\nAI 관제시스템개발\n➢다양한서비스를 보유하고 있는고객사를 위해실시간으로 시계열로그이\n상징후를 감지할수있는알고리즘 개발및운영을위한플랫폼을

In [ ]:
query = "파트너는 누구인가?"
result = qa({"query": query})

In [ ]:
result["result"]

' 휴먼플래닛이 인공지능팩토리의 파트너입니다.'

In [ ]:
result["source_documents"]

[Document(page_content='인공지능팩토리의파트너\n02\n휴먼플래닛', metadata={'source': 'docs/AF_회사소개서.pdf', 'page': 2}),
 Document(page_content='Company Profile\n인공지능팩토리회사소개서', metadata={'source': 'docs/AF_회사소개서.pdf', 'page': 0}),
 Document(page_content='05WHY AI FACTORY?전문성있는AI 컨설팅AI기술을도입하고싶지만, 어디부터어떻게적용해야할지막막한분들께인공지능팩토리는최고의파트너입니다. 업계전문가의수준높은컨설팅을통해, 문제정의부터최적화관리까지진정한의미의토탈솔루션을제공합니다.독보적인인공지능모델& 데이터평가검증역량인공지능팩토리는창업이후1년만에20개이상의고객사와함께경진대회개최및모델을개발검증해왔습니다. 다양한산업군경험과전문가기반의평가검증역량으로믿을수있는결과물을드립니다.마무리까지확실한유지보수관리인공지능팩토리는신뢰를중요시합니다. 단순히결과물을전달하는데서끝나지않고사후관리까지담당하여최적화된상태로인공지능모델이이용되도록합니다.', metadata={'source': 'docs/AF_회사소개서.pdf', 'page': 5}),
 Document(page_content='사업명 -\n과제명 입자결정 3차원분석모델및Crack 검출모델개발\n발주기관 LG화학주관기관 ㈜인공지능팩토리\n공동개발기관 -\n사업기간 ❖2021년03월15일~ 2021년09월01일\n사업목표❖[입자 3차원분석모델] 이온통로가되는음극재입자의벡터기반방향성정보\n를계량화하고 이를시각화하는 모듈개발\n❖[입자 Crack 검출] 셀의표본이미지로부터 정상입자와균열이발생한입자를\n탐지하여 계량적지표를산출할수있는 AI 모델개발\n사업내용❖입자 3차원분석모델개발\n➢이온통로가되는음극재입자의벡터기반방향성정보를계량화하고 이를\n시각화하는 모듈개발\n■3D 극성히스토그램 시각화모듈개발\n■벡터그룹이 3D 공간에서 가리키는 위치와선호